# Players Missed per Game due to Injury

In [62]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import commonallplayers

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

data_DIR = "../fdata/"
box_DIR = "../team_ratings/boxscores/"

In [58]:
def is_injured(dfinj, pId_missed, game_date):
    missed_games = np.array([False] * len(pId_missed))
    for i,pId in enumerate(pId_missed):
        df_p = dfinj.query(f'playerID == {pId}').reset_index(drop=True)
        if len(df_p) > 0:
            df_p["Comp"] = df_p["Date"] <= game_date
            idxi = df_p[df_p["Comp"]].index
            if len(idxi) > 0:
                idx = idxi[-1]
                missed_game = df_p["Out"].loc[idx]
                missed_games[i] = missed_game
    gp = missed_games*pId_missed
    pId_m = gp[gp !=0 ]
    list(pId_m)
    return pId_m

In [3]:
year = 2023

In [89]:
df0 = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{year}.csv")
df0= df0[["GAME_ID","TEAM_ID","GAME_DATE","MATCHUP","WL","PLUS_MINUS"]]

In [4]:
df1 = pd.read_parquet(data_DIR + "NBA_Player_BoxScores_" + "Indv" + "_" + str(year) + ".parquet")
df1["GAME_DATE"] = pd.to_datetime(df1["GAME_DATE"], format="%Y-%m-%d")

In [6]:
games_list = list(df1["GAME_ID"].unique())

In [7]:
dfinj = pd.read_parquet(data_DIR + f'NBA_prosptran_injuries_{year}.parquet')

In [36]:
dfr = commonallplayers.CommonAllPlayers(league_id = "00", season =year, is_only_current_season=True).get_data_frames()[0]
dfr = dfr.query("ROSTERSTATUS == 1 & GAMES_PLAYED_FLAG == 'Y'")
dfr = dfr.rename(columns={"PERSON_ID":"PLAYER_ID"})
dfr = dfr[["TEAM_ID","PLAYER_ID"]]
dfrt = dfr.groupby("TEAM_ID")

In [12]:
df2 = df1.groupby(["GAME_ID","TEAM_ID"])

In [13]:
keys = list(df2.groups)

In [66]:
dfma = []
for key in keys:
    p_played =  df2["PLAYER_ID"].get_group(key).to_list()
    p_roster = dfrt["PLAYER_ID"].get_group(key[1]).to_list()
    pId_missed = np.array(list(set(p_roster) - set(p_played))).astype(int)
    game_date = df2["GAME_DATE"].get_group(key).iloc[0]
    players_missed = is_injured(dfinj,pId_missed,game_date)
    players_missed
    dfm1 = pd.DataFrame({"PLAYER_ID":players_missed})
    dfm1["TEAM_ID"] = key[1]
    dfm1["GAME_ID"] = key[0]
    dfm1["GAME_DATE"] = game_date
    dfma.append(dfm1)
dfm = pd.concat(dfma)
dfm["PLAYER_ID"] = dfm["PLAYER_ID"].astype(int)

In [68]:
dfm["PLAYER_NAME"] = dfm["PLAYER_ID"].map(player_dict)
dfm["TEAM_NAME"] = dfm["TEAM_ID"].map(teams_dict)

In [127]:
dfm.query("TEAM_NAME == 'Memphis Grizzlies'").head(30)

,PLAYER_ID,TEAM_ID,GAME_ID,GAME_DATE,PLAYER_NAME,TEAM_NAME
0,1629634,1610612763,0022300007,2023-11-03,Brandon Clarke,Memphis Grizzlies
1,203500,1610612763,0022300007,2023-11-03,Steven Adams,Memphis Grizzlies
2,1630583,1610612763,0022300007,2023-11-03,Santi Aldama,Memphis Grizzlies
3,201565,1610612763,0022300007,2023-11-03,Derrick Rose,Memphis Grizzlies
0,1629634,1610612763,0022300012,2023-11-10,Brandon Clarke,Memphis Grizzlies
1,1630533,1610612763,0022300012,2023-11-10,Ziaire Williams,Memphis Grizzlies
2,1630214,1610612763,0022300012,2023-11-10,Xavier Tillman,Memphis Grizzlies
3,203500,1610612763,0022300012,2023-11-10,Steven Adams,Memphis Grizzlies
4,201565,1610612763,0022300012,2023-11-10,Derrick Rose,Memphis Grizzlies
0,1629634,1610612763,0022300026,2023-11-14,Brandon Clarke,Memphis Grizzlies


In [91]:
dfmg = dfm.groupby(["GAME_ID","TEAM_ID"])["PLAYER_ID"].count()
dfmg = pd.DataFrame(dfmg)
dfmg = dfmg.reset_index().rename(columns={"PLAYER_ID":"Players_Out"})
dfmg["GAME_ID"] = dfmg["GAME_ID"].astype(int)

In [124]:
dfmf = pd.merge(df0,dfmg,on=["GAME_ID","TEAM_ID"])
dfmf["Team"] = dfmf["TEAM_ID"].map(teams_dict)
dfmf = dfmf.drop(columns=["GAME_ID","TEAM_ID"])
dfmf.insert(1,"Team",dfmf.pop("Team"))

In [112]:
df_exp = dfmf.nlargest(10,"Players_Out")
df_exp = df_exp.reset_index(drop=True).reset_index()
df_exp["index"] += 1

In [113]:
df_exp

,index,GAME_DATE,Team,MATCHUP,WL,PLUS_MINUS,Players_Out
0,1,2023-11-06,Detroit Pistons,DET vs. GSW,L,-11,7
1,2,2023-11-08,Detroit Pistons,DET @ MIL,L,-2,6
2,3,2023-11-10,Detroit Pistons,DET vs. PHI,L,-8,6
3,4,2023-11-14,Charlotte Hornets,CHA vs. MIA,L,-6,6
4,5,2023-11-19,Memphis Grizzlies,MEM vs. BOS,L,-2,6
5,6,2023-11-19,Detroit Pistons,DET @ TOR,L,-29,6
6,7,2023-11-24,Phoenix Suns,PHX @ MEM,W,21,6
7,8,2023-11-25,Miami Heat,MIA @ BKN,L,-15,6
8,9,2023-12-02,Memphis Grizzlies,MEM @ PHX,L,-7,6
9,10,2023-12-16,Atlanta Hawks,ATL @ CLE,L,-8,6


In [110]:
%reload_ext rpy2.ipython

c:\Users\pansr\AppData\Local\pypoetry\Cache\virtualenvs\nba-ub9Z_EQq-py3.11\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.


In [120]:
%%R -i df_exp
library(tidyverse)
library(gt)
library(gtExtras)
df <- df_exp
df %>% 
  gt()%>%
  tab_header(
    title = md("**Most Players Out during a Game in 2023-24**"),
    subtitle = "Players Out: Players missing the game due to injury/personal reasons"  
    ) %>%
    data_color(columns = c(PLUS_MINUS), palette = c("red","green")) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(GAME_DATE,Team,MATCHUP))  %>%
    cols_label(
      index = "#",GAME_DATE = "Date", MATCHUP = "Matchup", PLUS_MINUS = "+/-", Players_Out ="Players Out"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(6)
    ) %>% 
    # tab_source_note(
    # source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats, prosportstransactions" ) %>% gtsave("../figs/team_leaders/players_missed.png",zoom=5) 

file:///C:/Users/pansr/AppData/Local/Temp/Rtmp46HdlL/file8805d01608f.html screenshot completed
